In [1]:
import os
import torch
import tarfile
import numpy as np

from torch import nn
from torch.nn import functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision import transforms as tt

torch.manual_seed(123)

In [15]:
# Function to calculate accuracy
def cal_accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# Function to evaluate model accuracy on data
@torch.no_grad()
def evaluate_model_data(model, test_loader):
    model.eval()
    model.eval()
    outputs = []
    for batch in test_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        out = model(images)                    
        loss = F.cross_entropy(out, labels)   
        acc = cal_accuracy(out, labels)
        outputs.append({'Loss': loss.detach(), 'Acc': acc})
    batch_losses = [x['Loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   
    batch_accs = [x['Acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()  
    return {'Loss': epoch_loss.item(), 'Acc': epoch_acc.item()}

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# Function to train the model on one epoch
def train_one_epoch(epochs, model, train_loader, test_loader):
    torch.cuda.empty_cache()
    history = []
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, weight_decay=1e-4)

    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    for epoch in range(epochs): 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            out = model(images)                  
            loss = F.cross_entropy(out, labels) 
            train_losses.append(loss)
            loss.backward()
            
            nn.utils.clip_grad_value_(model.parameters(), 0.1)
            
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            
        
        # Validation phase
        result = evaluate_model_data(model, test_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        print(f"Epoch {epoch} - train loss: {result['train_loss']:.4f}, test loss: {result['Loss']:.4f}, test accuracy: {result['Acc']:.4f}")
        history.append(result)
        sched.step(result['Loss'])
    return history

In [3]:
# Dowloading the CIFAR dataset
cifar_data_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(cifar_data_url, '.')

# Extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
cifar_data_dir = './data/cifar10'


100%|██████████| 135107811/135107811 [00:03<00:00, 37453617.45it/s]


In [4]:
# Normalizing the train and test data
normalize_train = tt.Compose([tt.ToTensor(), tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
normalize_test = tt.Compose([tt.ToTensor(), tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

In [5]:
train_data = ImageFolder(cifar_data_dir+'/train', normalize_train)
test_data = ImageFolder(cifar_data_dir+'/test', normalize_test)

In [6]:
batch_size = 256
train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=3, pin_memory=True)
test_loader = DataLoader(test_data, batch_size*2, num_workers=3, pin_memory=True)

In [16]:
device = "cuda:0"
base_model = resnet18(10).to(device = device)

In [17]:
# Training the base model
%%time
epochs = 40
history = train_one_epoch(epochs, base_model, train_loader, test_loader)
torch.save(base_model.state_dict(), "pretrained_resnet18.pt")

Epoch 0 - train loss: 1.8526, test loss: 8.6622, test accuracy: 0.3518
Epoch 1 - train loss: 1.3670, test loss: 1.1756, test accuracy: 0.5936
Epoch 2 - train loss: 1.2886, test loss: 1.2093, test accuracy: 0.5698
Epoch 3 - train loss: 1.0033, test loss: 1.0226, test accuracy: 0.6547
Epoch 4 - train loss: 0.8851, test loss: 0.9706, test accuracy: 0.6646
Epoch 5 - train loss: 0.7919, test loss: 0.8523, test accuracy: 0.7024
Epoch 6 - train loss: 0.7294, test loss: 0.9397, test accuracy: 0.6882
Epoch 7 - train loss: 0.6833, test loss: 0.9255, test accuracy: 0.6922
Epoch 8 - train loss: 0.6632, test loss: 1.0140, test accuracy: 0.6785
Epoch 9 - train loss: 0.6229, test loss: 0.8675, test accuracy: 0.7146
Epoch 00010: reducing learning rate of group 0 to 5.0000e-03.
Epoch 10 - train loss: 0.4414, test loss: 0.7626, test accuracy: 0.7504
Epoch 11 - train loss: 0.3953, test loss: 0.8481, test accuracy: 0.7284
Epoch 12 - train loss: 0.3669, test loss: 0.8267, test accuracy: 0.7429
Epoch 13 - t

In [18]:
# Calculating accuracy of test data on the base model
base_model.load_state_dict(torch.load("pretrained_resnet18.pt"))
evaluate_model_data(base_model, test_loader)


{'Loss': 1.5975735187530518, 'Acc': 0.7662626504898071}

In [19]:
# creating noise 
class Noise(nn.Module):
    def __init__(self, *dim):
        super().__init__()
        self.noise = torch.nn.Parameter(torch.randn(*dim), requires_grad = True)
        
    def forward(self):
        return self.noise

In [21]:
# list of all classes
all_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# classes which are to be un-learned
forget_classes = [0, 2]

# Retain classes
retain_classses = list(set(all_classes) - set(forget_classes))

In [22]:
# samples of all classes
class_train = {}
for i in all_classes:
    class_train[i] = []

for img, label in train_data:
    class_train[label].append((img, label))
    
class_test = {}
for i in all_classes:
    class_test[i] = []

for img, label in test_data:
    class_test[label].append((img, label))

In [23]:
# getting some samples from retain classes
num_samples_per_class = 1000

retain_samples = []
for i in range(len(all_classes)):
    if all_classes[i] in retain_classses:
        retain_samples += class_train[i][:num_samples_per_class]
        

In [24]:
# retain test set
retain_test = []
for cls in all_classes:
    if cls not in forget_classes:
        for img, label in class_test[cls]:
            retain_test.append((img, label))
            
# forget test set
forget_test = []
for cls in all_classes:
    if cls in forget_classes:
        for img, label in class_test[cls]:
            forget_test.append((img, label))
            
forget_test_loader = DataLoader(forget_test, batch_size, num_workers=3, pin_memory=True)
retain_test_loader = DataLoader(retain_test, batch_size*2, num_workers=3, pin_memory=True)

In [25]:
# loading the weights to the resnet18 unlearn model
unlearn_model = resnet18(10).to(device = device)
unlearn_model.load_state_dict(torch.load("pretrained_resnet18.pt"))

<All keys matched successfully>

In [26]:
%%time
# Maximizing noise for forget classes
noises = {}
for cls in forget_classes:
    print("Optiming loss for class {}".format(cls))
    noises[cls] = Noise(batch_size, 3, 32, 32).cuda()
    opt = torch.optim.Adam(noises[cls].parameters(), lr = 0.1)

    num_epochs = 5
    num_steps = 8
    class_label = cls
    for epoch in range(num_epochs):
        total_loss = []
        for batch in range(num_steps):
            inputs = noises[cls]()
            labels = torch.zeros(batch_size).cuda()+class_label
            outputs = unlearn_model(inputs)
            loss = -F.cross_entropy(outputs, labels.long()) + 0.1*torch.mean(torch.sum(torch.square(inputs), [1, 2, 3]))
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss.append(loss.cpu().detach().numpy())
        print("Loss: {}".format(np.mean(total_loss)))

Optiming loss for class 0
Loss: 192.3466796875
Loss: 41.73059844970703
Loss: 0.5920121669769287
Loss: -7.632769584655762
Loss: -11.03775691986084
Optiming loss for class 2
Loss: 192.3104248046875
Loss: 41.819068908691406
Loss: 0.6195129156112671
Loss: -7.725771903991699
Loss: -11.26416301727295
CPU times: user 6 s, sys: 21.9 ms, total: 6.02 s
Wall time: 6.03 s


In [27]:
%%time
# Minimizing noise for retain classes
for cls in retain_classses:
    print("Optiming loss for class {}".format(cls))
    noises[cls] = Noise(batch_size, 3, 32, 32).cuda()
    opt = torch.optim.Adam(noises[cls].parameters(), lr = 0.1)

    num_epochs = 5
    num_steps = 8
    class_label = cls
    for epoch in range(num_epochs):
        total_loss = []
        for batch in range(num_steps):
            inputs = noises[cls]()
            labels = torch.zeros(batch_size).cuda()+class_label
            outputs = unlearn_model(inputs)
            loss = F.cross_entropy(outputs, labels.long())
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss.append(loss.cpu().detach().numpy())
        print("Loss: {}".format(np.mean(total_loss)))

Optiming loss for class 1
Loss: 9.898370742797852
Loss: 6.415439605712891
Loss: 5.39954948425293
Loss: 4.83894157409668
Loss: 4.452078342437744
Optiming loss for class 3
Loss: 8.842272758483887
Loss: 5.321497440338135
Loss: 4.336767196655273
Loss: 3.8340373039245605
Loss: 3.461146593093872
Optiming loss for class 4
Loss: 9.927892684936523
Loss: 6.55422306060791
Loss: 5.4255290031433105
Loss: 4.8702239990234375
Loss: 4.4648308753967285
Optiming loss for class 5
Loss: 9.600830078125
Loss: 6.396271705627441
Loss: 5.394969463348389
Loss: 4.858287334442139
Loss: 4.482732772827148
Optiming loss for class 6
Loss: 9.979734420776367
Loss: 6.746089458465576
Loss: 5.737107753753662
Loss: 5.159703254699707
Loss: 4.749669551849365
Optiming loss for class 7
Loss: 10.753073692321777
Loss: 7.230379104614258
Loss: 6.165305137634277
Loss: 5.581254005432129
Loss: 5.17600154876709
Optiming loss for class 8
Loss: 9.351963996887207
Loss: 5.878281116485596
Loss: 4.796874046325684
Loss: 4.176741600036621
Loss

In [28]:
%%time

# Training the unlearn model with the noise data 
batch_size = 256
noisy_data = []
num_batches = 20
class_num = 0

for cls in forget_classes:
    for i in range(num_batches):
        batch = noises[cls]().cpu().detach()
        for i in range(batch[0].size(0)):
            noisy_data.append((batch[i], torch.tensor(class_num)))

noisy_loader = torch.utils.data.DataLoader(noisy_data, batch_size=256, shuffle = True)


optimizer = torch.optim.Adam(unlearn_model.parameters(), lr = 0.02)

epochs = 5
for epoch in range(epochs):  
    unlearn_model.train(True)
    running_loss = 0.0
    running_acc = 0
    for i, data in enumerate(noisy_loader):
        inputs, labels = data
        inputs, labels = inputs.cuda(),torch.tensor(labels).cuda()

        optimizer.zero_grad()
        outputs = unlearn_model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        out = torch.argmax(outputs.detach(),dim=1)
        assert out.shape==labels.shape
        running_acc += (labels==out).sum().item()

<timed exec>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


CPU times: user 12.2 s, sys: 77.5 ms, total: 12.2 s
Wall time: 12.3 s


In [31]:
print("Performance of unlearn Model on Forget Class")
results = evaluate_model_data(unlearn_model, forget_test_loader)
print("Accuracy: {}".format(results["Acc"]*100))
print("Loss: {}".format(results["Loss"]))

print("\nPerformance of unlearn Model on Retain Class")
results = evaluate_model_data(unlearn_model, retain_test_loader)
print("Accuracy: {}".format(results["Acc"]*100))
print("Loss: {}".format(results["Loss"]))

Performance of unlearn Model on Forget Class
Accuracy: 0.048828125
Loss: 12.01893424987793

Performance of unlearn Model on Retain Class
Accuracy: 66.97998046875
Loss: 1.2579243183135986
